In [ ]:
# 7.Création du pipeline
def create_pipeline(X_train):
    """Pipeline amélioré avec corrections des erreurs"""
    
    numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    
    numeric_transformer = Pipeline(steps=[
        ('imputer', IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=42)),
        ('scaler', StandardScaler()),
        ('feature_selection', SelectKBest(score_func=f_regression, k='all'))
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop'
    )
    
    models = {
        'Ridge': RidgeCV(alphas=np.logspace(-3, 3, 100)),
        'Lasso': LassoCV(alphas=np.logspace(-4, 2, 50), cv=5, random_state=42),
        'RandomForest': RandomForestRegressor(
            n_estimators=200,
            max_depth=10,
            min_samples_leaf=10,
            random_state=42,
            n_jobs=-1
        ),
        'LinearRegression': LinearRegression(),
        'DecisionTree': DecisionTreeRegressor(
            max_depth=6,
            min_samples_leaf=10,
            random_state=42
        ),
        'KNN': KNeighborsRegressor(n_neighbors=5),
        
        'HistGradientBoosting': HistGradientBoostingRegressor(
            max_iter=200,
            learning_rate=0.05,
            max_depth=3,
            min_samples_leaf=15,
            l2_regularization=0.2,
            random_state=42
        ),
        'XGBoost': XGBRegressor(
            n_estimators=300,
            max_depth=4,
            learning_rate=0.05,
            reg_alpha=0.2,
            reg_lambda=1.0,
            early_stopping_rounds=10,
            random_state=42
        )
    }
    
    # 5. Construction des pipelines optimisés
    pipelines = {}
    for name, model in models.items():
        steps = [
            ('preprocessor', preprocessor),
            ('model', model)
        ]
        
        # Ajout de RFECV seulement pour les modèles linéaires
        if name in ['Ridge', 'Lasso', 'LinearRegression']:
            steps.insert(1, ('feature_selector', RFECV(
                estimator=model,
                step=5,
                cv=3,
                scoring='neg_mean_squared_error',
                n_jobs=-1
            )))
        
        pipelines[name] = Pipeline(steps)
    return pipelines, preprocessor, numeric_features, categorical_features

In [ ]:
# 8.Evaluation et optimisation des modèles
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def evaluate_and_optimize_models(pipelines, X_train, X_test, y_train, y_test):
    """
    Évalue et optimise les modèles avec gestion spéciale pour XGBoost,
    correction pour véhicules premium et comparaison des performances.
    """
    all_results = {}
    results = []
    
    print("📊 Début de l'évaluation des modèles...")

    for name, pipeline in pipelines.items():
        print(f"\n🔍 Traitement du modèle {name}...")
        
        try:
            # Gestion spéciale pour XGBoost
            if 'XGB' in name or 'XGBoost' in name:
                # 1. Créer une copie du preprocessor
                preprocessor = clone(pipeline.named_steps['preprocessor'])
                
                # 2. Préparer les données transformées
                X_train_transformed = preprocessor.fit_transform(X_train, y_train)
                X_test_transformed = preprocessor.transform(X_test)
                
                # 3. Configurer XGBoost avec early stopping
                model = XGBRegressor(
                    n_estimators=1000,  # Nombre élevé car early stopping activé
                    max_depth=5,
                    learning_rate=0.05,
                    reg_alpha=0.1,
                    reg_lambda=1.0,
                    early_stopping_rounds=50,
                    random_state=42,
                    n_jobs=-1
                )
                
                # 4. Entraînement avec jeu de validation
                model.fit(
                    X_train_transformed, y_train,
                    eval_set=[(X_test_transformed, y_test)],
                    verbose=10  # Affiche les métriques toutes les 10 itérations
                )
                
                # 5. Reconstruire le pipeline complet
                pipeline.named_steps['preprocessor'] = preprocessor
                pipeline.named_steps['model'] = model
            else:
                # Entraînement standard pour les autres modèles
                pipeline.fit(X_train, y_train)
            
                # Entraînement du modèle
                pipeline.fit(X_train, y_train)
            
                # Prédictions de base
                y_train_pred = pipeline.predict(X_train)
                y_test_pred = pipeline.predict(X_test)
            
                
            # Calcul des métriques
            metrics = {
                'Modèle': name,
                'MAPE_train': mean_absolute_percentage_error(y_train, y_train_pred),
                'MAPE_test': mean_absolute_percentage_error(y_test, y_test_pred),
                'RMSE_train': np.sqrt(mean_squared_error(y_train, y_train_pred)),
                'MAE_train': mean_absolute_error(y_train, y_train_pred),
                'R²_train': r2_score(y_train, y_train_pred),
                'RMSE_test': np.sqrt(mean_squared_error(y_test, y_test_pred)),
                'MAE_test': mean_absolute_error(y_test, y_test_pred),
                'R²_test': r2_score(y_test, y_test_pred),
                'Type': type(pipeline.named_steps['model']).__name__
            }
            
            # Stockage des résultats
            results.append(metrics)
            all_results[name] = metrics
            
            print(f"✅ {name} - R² Test: {metrics['R²_test']:.3f} | RMSE Test: {metrics['RMSE_test']:.2f}")
            
        except Exception as e:
            print(f"❌ Erreur avec {name}: {str(e)}")
            continue
    
    # Création du DataFrame des résultats
    results_df = pd.DataFrame(results)
    
    # Calcul du score agrégé (plus bas = meilleur)
    results_df['Score_agrégé'] = (
        results_df['RMSE_test'] + 
        results_df['MAE_test'] - 
        results_df['R²_test']
    )
    
    # Sélection du meilleur modèle
    best_idx = results_df['Score_agrégé'].idxmin()
    best_model_name = results_df.loc[best_idx, 'Modèle']
    best_model_final = pipelines[best_model_name]
    
    # Affichage des résultats
    print("\n📋 Résultats détaillés :")
    print(results_df.sort_values('Score_agrégé').to_string(index=False))
    
    # Visualisation
    plot_model_comparisons(results_df)
    
    return all_results, best_model_name, best_model_final

def plot_model_comparisons(results_df):
    """Visualisation comparative des performances des modèles"""
    plt.figure(figsize=(15, 8))
    
    # Tri des résultats par R² test
    results_df = results_df.sort_values('R²_test', ascending=False)
    
    # Graphique à barres pour R²
    plt.subplot(2, 2, 1)
    plt.barh(results_df['Modèle'], results_df['R²_test'], color='skyblue')
    plt.title('Comparaison des R² (Test)')
    plt.xlim(0, 1)
    
    # Graphique à barres pour RMSE
    plt.subplot(2, 2, 2)
    plt.barh(results_df['Modèle'], results_df['RMSE_test'], color='lightgreen')
    plt.title('Comparaison des RMSE (Test)')
    
    # Graphique à barres pour MAE
    plt.subplot(2, 2, 3)
    plt.barh(results_df['Modèle'], results_df['MAE_test'], color='salmon')
    plt.title('Comparaison des MAE (Test)')
    
    # Graphique combiné
    plt.subplot(2, 2, 4)
    width = 0.3
    x = np.arange(len(results_df))
    plt.bar(x - width, results_df['R²_test'], width, label='R²')
    plt.bar(x, results_df['RMSE_test'], width, label='RMSE')
    plt.bar(x + width, results_df['MAE_test'], width, label='MAE')
    plt.xticks(x, results_df['Modèle'], rotation=45)
    plt.legend()
    plt.title('Métriques combinées')
    
    plt.tight_layout()
    plt.show()